In [ ]:
# Structure is the same as in LatticeAllHCPTaskPredNegative except that the task maps have been inverted (see code block 5, line 18)

import rpy2
print(rpy2.__version__)

In [ ]:
from rpy2.robjects.packages import importr
# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

packnames = ('ggplot2', 'spam64')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [ ]:

import nibabel as nb
import numpy as np
 
# cifti code based on Christopher J Markiewicz https://nbviewer.jupyter.org/github/neurohackademy/nh2020-curriculum/blob/master/we-nibabel-markiewicz/NiBabel.ipynb
# load cifti file containing the vertexwise values, in this case myelin map from the HCP
cifti = nb.load('HCP_S900_787_tfMRI_ALLTASKS_level3_zstat1_hp200_s2_MSMSulc.dscalar.nii')
cifti_data = cifti.get_fdata(dtype=np.float32)
cifti_hdr = cifti.header
nifti_hdr = cifti.nifti_header

axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]

def surf_data_from_cifti(data, axis, surf_name): 
    assert isinstance(axis, nb.cifti2.BrainModelAxis)
    for name, data_indices, model in axis.iter_structures():  # Iterates over volumetric and surface structures
        if name == surf_name:                                 # Just looking for a surface
            data = data.T[data_indices]                       # Assume brainmodels axis is last, move it to front
            vtx_indices = model.vertex                        # Generally 1-N, except medial wall vertices
            surf_data = np.zeros((vtx_indices.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vtx_indices] = data
            return surf_data
    raise ValueError(f"No structure named {surf_name}")

left_brain=surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT')

#load gifti surface that is the same resolution/underlying mesh etc as the cifti
gifti_img_BaseBrain = nb.load('S900.L.sphere.32k_fs_LR.surf.gii')



xyz_points=gifti_img_BaseBrain.darrays[0].data



In [ ]:
import numpy as np

def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    
    xy = xyz[:,0]**2 + xyz[:,1]**2
    ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
    #ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
    ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew

ptsnew=appendSpherical_np(xyz_points)

In [ ]:
import numpy as np
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector
from rpy2.robjects import r, pandas2ri
import pandas as pd
import scipy


pandas2ri.activate()
LK = importr('LatticeKrig')
cifti = nb.load('HCP_S900_787_tfMRI_ALLTASKS_level3_zstat1_hp200_s2_MSMSulc.dscalar.nii')
cifti_data = cifti.get_fdata(dtype=np.float32)
cifti_hdr = cifti.header
nifti_hdr = cifti.nifti_header

axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]
left_brain=surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT')
left_brain=left_brain*-1
taskNames=np.loadtxt('TaskNames.txt',dtype=str,delimiter=',')
tasks=np.asarray([9,31,42,62,69,74,80])

LKPredAllTasks=np.zeros([ptsnew.shape[0],tasks.shape[0]])
AllThresh=np.zeros([7])
count=0
for i in range(tasks.shape[0]):

        task=tasks[i]
        threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
        threshLower= np.percentile(left_brain[:,task], 25, axis=0)

        AllThresh[i]=threshUpper

        pos_index=left_brain[:,task]>threshUpper
        neg_index=left_brain[:,task]<threshLower

        X_temp=np.degrees(ptsnew[pos_index,4])

        X1=FloatVector(X_temp[:])

        X_temp=np.degrees(ptsnew[pos_index,5])

        X2=FloatVector(X_temp[:])
        Y_temp=left_brain[pos_index,task]

        Y=FloatVector(Y_temp[:])


        df = pd.DataFrame({'X1': X2, 'X2': X1})

        r_dataframe = pandas2ri.py2rpy(df)
        alpha=FloatVector(np.power([1,0.5,0.1],2))

        LKinfo=LK.LKrigSetup(r_dataframe,startingLevel=3,nlevel=3,a_wght=1.01,alpha=alpha,LKGeometry="LKSphere",Radius=100) 

        LKOutput=LK.LatticeKrig(r_dataframe,Y,LKinfo=LKinfo)

        X1_all=FloatVector(np.degrees(ptsnew[:,4]))
        X2_all=FloatVector(np.degrees(ptsnew[:,5]))
        Y_all=FloatVector(left_brain[:,task])


        df_all = pd.DataFrame({'X1': X2_all, 'X2': X1_all})

        r_dataframe_all = pandas2ri.py2rpy(df_all)
        LKPredAllTasks[:,count]=r.predict(LKOutput,r_dataframe_all).ravel()
        count=count+1


In [ ]:
import matplotlib.pyplot as plt

gifti_img_Midthickness = nb.load('S900.L.midthickness_MSMAll.32k_fs_LR.surf.gii')
xyz_points_Mid=gifti_img_Midthickness.darrays[0].data
azim=0


TPProj=(LKPredAllTasks<0).mean(axis=1)


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
p=ax.scatter(xyz_points_Mid[np.abs(left_brain[:,0])>0,0],xyz_points_Mid[np.abs(left_brain[:,0])>0,1],xyz_points_Mid[np.abs(left_brain[:,0])>0,2],c=TPProj[np.abs(left_brain[:,0])>0],s=3,cmap='Reds',vmin=0,vmax=1)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)

TPReal=(left_brain[:,tasks]<0).mean(axis=1)
TPReal_NoMedialWall=np.where(TPReal==0,0,TPReal)


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[np.abs(left_brain[:,0])>0,0],xyz_points_Mid[np.abs(left_brain[:,0])>0,1],xyz_points_Mid[np.abs(left_brain[:,0])>0,2],c=TPReal[np.abs(left_brain[:,0])>0],s=3,cmap='Reds',vmin=0,vmax=1)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)

TPProj_NoMedialWall=np.where(TPReal==0,0,TPProj)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[np.abs(left_brain[:,0])>0,0],xyz_points_Mid[np.abs(left_brain[:,0])>0,1],xyz_points_Mid[np.abs(left_brain[:,0])>0,2],c=TPReal[np.abs(left_brain[:,0])>0]-TPProj[np.abs(left_brain[:,0])>0],s=3,cmap='Reds',vmin=0,vmax=1)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)




In [ ]:
TPProj_NoMedialWall

In [ ]:
az=0

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=fcgradient[:,0],s=3)

ax.view_init(elev=10., azim=az)
fig.colorbar(p,ax=ax)
plt.show()


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=fcgradient[:,1],s=3)

ax.view_init(elev=10., azim=az)
fig.colorbar(p,ax=ax)
plt.show()
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=fcgradient[:,2],s=3)

ax.view_init(elev=10., azim=az)
fig.colorbar(p,ax=ax)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=TPProj_NoMedialWall,s=3)
ax.view_init(elev=10., azim=az)
fig.colorbar(p,ax=ax)

In [ ]:

azim=180
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
p=ax.scatter(xyz_points_Mid[:,0],xyz_points_Mid[:,1],xyz_points_Mid[:,2],c=TPProj_NoMedialWall,s=3,cmap='Reds',vmin=0,vmax=1)
ax.view_init(elev=10., azim=azim)
fig.colorbar(p,ax=ax)


In [ ]:
TaskProjCompareTasks=np.zeros([7,7])

#import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
num_tasks=tasks.shape[0]

fig, axes = plt.subplots(1,2, figsize=(5* 2 , 5))

for taskID, task in enumerate(tasks):
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    
    pos_index=left_brain[:,task]>threshUpper
    neg_index=left_brain[:,task]<threshUpper

    for taskID2, task2 in enumerate(tasks):
        predtask=LKPredAllTasks[neg_index, taskID]*-1
        task_data = left_brain[neg_index, task2]*-1
        TaskProjCompareTasks[taskID,taskID2]=stats.spearmanr(predtask,task_data)[0]
        
TaskProjCompareTasks=TaskProjCompareTasks[::-1,:]
axes[0].pcolormesh((TaskProjCompareTasks), edgecolors='k', linestyle=':',linewidth=2,vmin=-1,vmax=1,cmap='bwr')

axes[0].tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False)



TaskProjCompareTasks=np.zeros([7,7])

#import seaborn as sns
from scipy import stats
num_tasks=tasks.shape[0]

count=0
for taskID, task in enumerate(tasks):
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    

    pos_index=left_brain[:,task]>threshUpper
    neg_index=left_brain[:,task]<0

    for taskID2, task2 in enumerate(tasks):
        predtask=LKPredAllTasks[neg_index, taskID]*-1
        task_data = left_brain[neg_index, task2]*-1
        TaskProjCompareTasks[taskID,taskID2]=stats.spearmanr(predtask,task_data)[0]
        
TaskProjCompareTasks=TaskProjCompareTasks[::-1,:]
axes[1].pcolormesh((TaskProjCompareTasks), edgecolors='k', linestyle='-',linewidth=2,vmin=-1,vmax=1,cmap='bwr',shading='flat')

#plt.savefig("TaskIntersectionPermuteTPRestrict.png", bbox_inches='tight')
axes[1].tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False) 


In [ ]:
TaskProjCompareTasks=np.zeros([7,7])

#import seaborn as sns
from scipy import stats
num_tasks=tasks.shape[0]

fig, axes = plt.subplots(1,2, figsize=(5* 2 , 5))

for taskID, task in enumerate(tasks):
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    
    pos_index=left_brain[:,task]>threshUpper
    neg_index=left_brain[:,task]<threshUpper

    for taskID2, task2 in enumerate(tasks):
        predtask=LKPredAllTasks[neg_index, taskID]*-1
        predtask2=LKPredAllTasks[neg_index, taskID2]*-1
        TaskProjCompareTasks[taskID,taskID2]=stats.spearmanr(predtask,predtask2)[0]
        
TaskProjCompareTasks=TaskProjCompareTasks[::-1,:]
axes[0].pcolormesh((TaskProjCompareTasks), edgecolors='k', linestyle=':',linewidth=2,vmin=-1,vmax=1,cmap='bwr')

axes[0].tick_params(left = False, right = False , labelleft = False , 
                labelbottom = False, bottom = False)

plt.savefig('PredSimilarityAcrossTasks.png')


In [ ]:
np.save('LKPredAllTasksPos',LKPredAllTasks)

In [ ]:
from neuromaps import datasets
from neuromaps.datasets import available_annotations,fetch_annotation
import numpy as np
import matplotlib.pyplot as plt

fslr = datasets.fetch_atlas(atlas='fslr', density='32k')
print(fslr.keys())

import nibabel as nib
lsphere, rsphere = fslr['sphere']
#lsphere, rsphere = fslr['midthickness']
lvert, ltri = nib.load(lsphere).agg_data()
print(lvert.shape, ltri.shape)

fcgradient=np.zeros([lvert.shape[0],10])

for i in range(10):
    grad_name='fcgradient'+ str(i+1).zfill(2)
    annotation = fetch_annotation(source='margulies2016',hemi='L',den='32k',desc=grad_name,return_single=True)
    fcgradient[:,i]=nib.load(annotation).agg_data()


CorrelationGradToTask=np.zeros([7,3])
CorrelationGradToPrediction=np.zeros([7,3])
CorrelationGradToOverlap=np.zeros([10])


for taskID, task in enumerate(tasks):
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    pos_index=left_brain[:,task]>threshUpper
    neg_index=left_brain[:,task]<threshUpper

    for grad in range(3):
        CorrelationGradToTask[taskID,grad]=stats.spearmanr(fcgradient[:,grad],left_brain[:,task])[0]
        CorrelationGradToPrediction[taskID,grad]=stats.spearmanr(fcgradient[pos_index,grad],LKPredAllTasks[pos_index,taskID])[0]
        
for grad in range(10):

    CorrelationGradToOverlap[grad]=stats.spearmanr(TPProj_NoMedialWall[np.abs(fcgradient[:,grad])>0],fcgradient[np.abs(fcgradient[:,grad])>0,grad])[0]

    
    

In [ ]:
import statsmodels.api as sm
Y = TPProj_NoMedialWall[np.abs(fcgradient[:,grad])>0]
X = fcgradient[np.abs(fcgradient[:,grad])>0,:10]
X = sm.add_constant(X)
model = sm.OLS(Y,X)

results = model.fit()
#results.tvalues
results.params


CorrelationGradToOverlap

In [ ]:

plt.scatter(TaskProjCompareTasks.flatten()[TaskProjCompareTasks.flatten()<0.99999],differences.flatten()[TaskProjCompareTasks.flatten()<0.99999])

stats.pearsonr(TaskProjCompareTasks.flatten()[TaskProjCompareTasks.flatten()<0.99999],differences.flatten()[TaskProjCompareTasks.flatten()<0.99999])

In [ ]:



from scipy.spatial import cKDTree
import numpy as np

YeoNetworks=np.zeros([left_brain.shape[0],7])
for i in range(7):
    cifti = nb.load('Yeo/7Networks' + str(i+1) + '.dscalar.nii')
    cifti_data = cifti.get_fdata(dtype=np.float32)
    cifti_hdr = cifti.header
    nifti_hdr = cifti.nifti_header
    axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]
    left_brainYeo=surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT')
    #left_brainYeoDMN=surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_RIGHT')
    YeoNetworks[:,i]=left_brainYeo[:,0]

realProjYeo = np.zeros([7])
# taskToDelete=0
# matrix=differences.copy()
# matrix = np.delete(matrix, taskToDelete, axis=0)
# matrix = np.delete(matrix, taskToDelete, axis=1)
# plt.imshow(matrix,cmap='bwr',vmin=-0.5,vmax=0.5)
# plt.colorbar()
                    
#plt.scatter(TPProj_NoMedialWall[np.abs(fcgradient[:,0])>0],TPReal_NoMedialWall[np.abs(fcgradient[:,0])>0])
for yn in range(7):
    realProjYeo[yn]=stats.spearmanr(TPProj_NoMedialWall[YeoNetworks[:,yn]==1],TPReal_NoMedialWall[YeoNetworks[:,yn]==1])[0]




In [ ]:
def write_plyRGB(filename, vertices, faces, colorsR,colorsG,colorsB,comment=None):
    import pandas as pd
    print("writing ply format")
    # infer number of vertices and faces
    number_vertices = vertices.shape[0]
    number_faces = faces.shape[0]
    # make header dataframe
    header = ['ply',
            'format ascii 1.0',
            'comment %s' % comment,
            'element vertex %i' % number_vertices,
            'property float x',
            'property float y',
            'property float z',
            'property uchar red',
            'property uchar green',
            'property uchar blue',
            'element face %i' % number_faces,
            'property list uchar int vertex_indices',
            'end_header'
             ]
    header_df = pd.DataFrame(header)
    # make dataframe from vertices
    vertex_df = pd.DataFrame(vertices/50)
    #colors_df = pd.DataFrame(np.tile(np.round(colors/7*255), (3,1)).T)
    ColorsR_df=pd.DataFrame(colorsR)
    ColorsG_df=pd.DataFrame(colorsG)
    ColorsB_df=pd.DataFrame(colorsB)
    colorsConcat = pd.concat([ColorsR_df,ColorsG_df,ColorsB_df], axis=1)
    colors_df=pd.DataFrame(colorsConcat)
    colors_df=colorsConcat.astype(int)
    df_concat = pd.concat([vertex_df, colors_df], axis=1)
    # make dataframe from faces, adding first row of 3s (indicating triangles)
    triangles = np.reshape(3 * (np.ones(number_faces)), (number_faces, 1))
    triangles = triangles.astype(int)
    faces = faces.astype(int)
    faces_df = pd.DataFrame(np.concatenate((triangles, faces), axis=1))
    # write dfs to csv
    header_df.to_csv(filename, header=None, index=False)
    with open(filename, 'a') as f:
        df_concat.to_csv(f, header=False, index=False,
                         float_format='%.3f', sep=' ')
    with open(filename, 'a') as f:
        faces_df.to_csv(f, header=False, index=False,
                        float_format='%.0f', sep=' ')

In [ ]:
import  matplotlib.cm
cmap = matplotlib.cm.get_cmap('Reds')



vertices=gifti_img_Midthickness.darrays[0].data
faces=gifti_img_Midthickness.darrays[1].data
colors=TPProj_NoMedialWall#/7#*254+1
CMap=cmap(colors)*255
print(CMap)

write_plyRGB('TPProj.ply',vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

vertices=gifti_img_Midthickness.darrays[0].data
faces=gifti_img_Midthickness.darrays[1].data
colors=TPReal_NoMedialWall#/7#*254+1
CMap=cmap(colors)*255


write_plyRGB('TPReal.ply',vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

cmap = matplotlib.cm.get_cmap('coolwarm')
vertices=gifti_img_Midthickness.darrays[0].data
faces=gifti_img_Midthickness.darrays[1].data
colors=TPReal_NoMedialWall-TPProj_NoMedialWall#/7#*254+1
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
CMap=cmap(norm(colors))*255


write_plyRGB('TPDiff.ply',vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])


In [ ]:

cmap = matplotlib.cm.get_cmap('coolwarm_r')

for i, task in enumerate(tasks):
    vertices=gifti_img_Midthickness.darrays[0].data
    faces=gifti_img_Midthickness.darrays[1].data    
    print(task)
    print(taskNames[task])
    
    colors=left_brain[:,task]
    #tempColors=np.where(TPReal==0,0,colors)
    norm = matplotlib.colors.Normalize(vmin=-20, vmax=20)
    CMap=cmap(norm(colors))*255
    PlyFileName='TPTaskActivity' + taskNames[task] + '.ply'
    write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])
 
    colors=np.zeros(left_brain[:,task].shape)
    colors[left_brain[:,task]>AllThresh[i]]=left_brain[left_brain[:,task]>AllThresh[i],task]
    norm = matplotlib.colors.Normalize(vmin=-20, vmax=20)
    CMap=cmap(norm(colors))*255
    PlyFileName='TPThresholded' + taskNames[task] + '.ply'
    write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])

    
    tempColors=np.where(left_brain[:,task]==0,0,LKPredAllTasks[:,i])
    #colors=LKPredAllTasks[:,i]
    colors=tempColors.copy()
    norm = matplotlib.colors.Normalize(vmin=-20, vmax=20)
    CMap=cmap(norm(colors))*255
    PlyFileName='TPProjected' + taskNames[task] + '.ply'
    write_plyRGB(PlyFileName,vertices,faces,CMap[:,0],CMap[:,1],CMap[:,2])



In [ ]:
sns.set(font_scale=1.5)
sns.set_style('white') 

def customize_plot(ax,s_r):
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel=None)
    ax.set(ylabel=None)
    ax.set_title(f"Correlation = {s_r}")
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    ax.axhline(0, color='black', linestyle='--', linewidth=0.5)
    ax.axvline(0, color='black', linestyle='--', linewidth=0.5)

fig, axes = plt.subplots(1, num_tasks, figsize=(5.5 * num_tasks, 5))
count = 0
for taskID, task in enumerate(tasks):
    ax = axes[count]

    threshUpper = np.percentile(left_brain[:, task], 75, axis=0)
    pos_index = left_brain[:, task] > threshUpper
    neg_index = left_brain[:, task] < threshUpper

    
    clust_data = np.ravel(LKPredAllTasks[neg_index, taskID])*-1
    task_data = np.ravel(left_brain[neg_index, task])*-1
    
    
    s_r=format(stats.spearmanr(clust_data,task_data)[0], '.2f')
    df = pd.DataFrame({"LKPredAllTasks": clust_data, "left_brain": task_data})
    sns.kdeplot(data=df, x="LKPredAllTasks", y="left_brain", fill=True, ax=ax, cmap="gray_r",bw_adjust=0.3,levels=10)

    customize_plot(ax,s_r)
    count += 1

plt.savefig('EachTaskScatterRealPred_lt_75pcPredPos.png')

fig2, axes = plt.subplots(1, num_tasks, figsize=(5.5 * num_tasks, 5))
count = 0
for taskID, task in enumerate(tasks):
    ax = axes[count]

    neg_index = left_brain[:, task] < 0

    clust_data = np.ravel(LKPredAllTasks[neg_index, taskID]) * -1
    task_data = np.ravel(left_brain[neg_index, task]) * -1
    
    s_r=format(stats.spearmanr(clust_data,task_data)[0], '.2f')
    df = pd.DataFrame({"LKPredAllTasks": clust_data, "left_brain": task_data})
    sns.kdeplot(data=df, x="LKPredAllTasks", y="left_brain", fill=True, ax=ax, cmap="gray_r",bw_adjust=0.3,levels=10)
    
    customize_plot(ax,s_r)
    count += 1

plt.savefig('EachTaskScatterRealPred_ltPrePos_0.png')

plt.show()

In [ ]:
from sklearn.metrics import DistanceMetric

num_tasks=tasks.shape[0]
fig, axes = plt.subplots(1,num_tasks, figsize=(5.5* num_tasks , 5))
count=0
for i in range(len(tasks)):
    ax = axes[count]
    taskID=i
    task=tasks[taskID]
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    threshLower= np.percentile(left_brain[:,task], 25, axis=0)
    pos_index=left_brain[:,task]>threshUpper
    dist = DistanceMetric.get_metric('haversine')

    DistsExt=dist.pairwise(ptsnew[pos_index,4:6],ptsnew[pos_index==False,4:6])
    #DistsInt=dist.pairwise(ptsnew[pos_index,4:6])
    MinDistsExt=DistsExt.min(axis=0)
    sns.kdeplot(data=MinDistsExt,ax=ax,fill=True,cmap="gray_r")
    ax.set_xlim(xmin=0, xmax=0.6)
    ax.set_ylim(ymin=0, ymax=4.5)
    #ax.set_xlabel("Min distance")
    #ax.set_ylabel("Count")
    #ax.set_title(f"Task {taskNames[tasks[taskID]]}")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel=None)
    ax.set(ylabel=None)

    count=count+1

plt.savefig('EachTaskMinDistToPostThresh_PredPos.png')


In [ ]:
from sklearn.metrics import DistanceMetric

num_tasks=tasks.shape[0]
fig, axes = plt.subplots(1,num_tasks, figsize=(5.5* num_tasks , 5))
count=0
for i in range(len(tasks)):
    ax = axes[count]
    taskID=i
    task=tasks[taskID]
    threshUpper= np.percentile(left_brain[:,task], 75, axis=0)
    threshLower= np.percentile(left_brain[:,task], 25, axis=0)
    pos_index=left_brain[:,task]>threshUpper
    dist = DistanceMetric.get_metric('haversine')

    DistsExt=dist.pairwise(ptsnew[pos_index,4:6],ptsnew[pos_index==False,4:6])
    #DistsInt=dist.pairwise(ptsnew[pos_index,4:6])
    MinDistsExt=DistsExt.min(axis=0)

    count=count+1

plt.savefig('EachTaskMinDistToPostThresh_PredPos.png')
